<a href="https://colab.research.google.com/github/kyrajeep/DL_Projects/blob/master/generate_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#Code in the beginning from:https://medium.com/ai-advances/how-to-use-instantid-to-generate-identity-preserving-images-f6fe126277fc
#install dependencies
!pip install diffusers insightface onnxruntime accelerate

# Clone the InstantID repo to get the StableDiffusionXLInstantIDPipeline definition
!git clone https://github.com/InstantID/InstantID.git
!mv InstantID/pipeline_stable_diffusion_xl_instantid.py pipeline_stable_diffusion_xl_instantid.py
!mv InstantID/ip_adapter/ .

# import librairies and download pretrained models from HuggingFace
from huggingface_hub import hf_hub_download
import diffusers
from diffusers.models import ControlNetModel
from diffusers.utils import load_image
import torch
from pipeline_stable_diffusion_xl_instantid import StableDiffusionXLInstantIDPipeline, draw_kps

hf_hub_download(repo_id="InstantX/InstantID",
        filename="ControlNetModel/config.json", local_dir="./checkpoints")
hf_hub_download(repo_id="InstantX/InstantID",
        filename="ControlNetModel/diffusion_pytorch_model.safetensors",
        local_dir="./checkpoints")

controlnet = ControlNetModel.from_pretrained(
                './checkpoints/ControlNetModel',
                torch_dtype=torch.float16)
base_model = 'wangqixun/YamerMIX_v8'



fatal: destination path 'InstantID' already exists and is not an empty directory.
mv: cannot stat 'InstantID/pipeline_stable_diffusion_xl_instantid.py': No such file or directory
mv: cannot stat 'InstantID/ip_adapter/': No such file or directory


In [12]:
# define the pipeline
model = StableDiffusionXLInstantIDPipeline.from_pretrained(
    base_model,
    controlnet=controlnet,
    torch_dtype=torch.float16
)


The config attributes {'controlnet_list': ['controlnet', 'RPMultiControlNetModel'], 'requires_aesthetics_score': False} were passed to StableDiffusionXLInstantIDPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'controlnet_list': ['controlnet', 'RPMultiControlNetModel'], 'requires_aesthetics_score': False} are not expected by StableDiffusionXLInstantIDPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
cuda_available = torch.cuda.is_available()
device = torch.device('cuda') if cuda_available else torch.device('cpu')
#x = x.to(device)
model = model.to(device)

In [14]:
# The adaptor provides constraints in the generation process.
hf_hub_download(repo_id="InstantX/InstantID", filename="ip-adapter.bin", local_dir="./checkpoints")

# load the adapter in the pipeline
model.load_ip_adapter_instantid('./checkpoints/ip-adapter.bin')

In [15]:
! pip install zipfile36
import zipfile

In [38]:

# download the 'antelopev2' under ./models
!mkdir -p ./models
!gdown --id 18wEUfMNohBJ4K3Ly5wpTejPfDzp-8fI8 -O ./models/antelopev2.zip

with zipfile.ZipFile('models/antelopev2.zip','r') as zip_ref:
  zip_ref.extractall('models')

from insightface.app import FaceAnalysis

app = FaceAnalysis(name='antelopev2', root='./', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0.0, det_size=(640.0, 640.0))



/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=18wEUfMNohBJ4K3Ly5wpTejPfDzp-8fI8
From (redirected): https://drive.google.com/uc?id=18wEUfMNohBJ4K3Ly5wpTejPfDzp-8fI8&confirm=t&uuid=ea8bad26-dbf3-40fe-947a-1fd92fee4e91
To: /content/models/antelopev2.zip
100% 361M/361M [00:05<00:00, 64.9MB/s]


/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/glintr100.onnx recognition ['None', 3, 112, 112] 127.5 127.5
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/scrfd_10g_bnkps.onnx detection [1, 3, '?', '?'] 127.5 128.0
set det-size: (640.0, 640.0)


In [25]:
!pip install opencv-python
import cv2
import numpy as np

In [26]:
# load an image
face_image = load_image("drive/MyDrive/generate_faces/profile_for_diffusion.jpg")

# extract the face info
face_info = app.get(cv2.cvtColor(np.array(face_image), cv2.COLOR_RGB2BGR))
face_info = sorted(face_info, key=lambda x:(x['bbox'][2]-x['bbox'][0])*x['bbox'][3]-x['bbox'][1])[-1]

/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [46]:
'''
!ls
!cd ..
!ls
'''
print(type(face_info))
print([face_info])

<class 'insightface.app.common.Face'>
[{'bbox': array([ 828.91974,  656.42944, 1290.014  , 1288.1696 ], dtype=float32), 'kps': array([[ 962.4718 ,  883.1114 ],
       [1184.1234 ,  887.6867 ],
       [1078.8131 , 1053.6796 ],
       [ 984.80884, 1134.5159 ],
       [1151.2792 , 1138.4111 ]], dtype=float32), 'det_score': 0.798157, 'landmark_3d_68': array([[ 8.06796326e+02,  8.60794556e+02,  3.23116943e+02],
       [ 8.13768860e+02,  9.32287598e+02,  3.25096497e+02],
       [ 8.23500488e+02,  1.00503387e+03,  3.22267914e+02],
       [ 8.35825439e+02,  1.07401465e+03,  3.10785095e+02],
       [ 8.60057739e+02,  1.13865430e+03,  2.71869934e+02],
       [ 8.93830017e+02,  1.18978430e+03,  2.34650192e+02],
       [ 9.33867126e+02,  1.23166309e+03,  2.00261703e+02],
       [ 9.84614807e+02,  1.26797351e+03,  1.55057495e+02],
       [ 1.05654053e+03,  1.29517468e+03,  1.42768753e+02],
       [ 1.12558374e+03,  1.27421069e+03,  1.66785416e+02],
       [ 1.17549683e+03,  1.23422388e+03,  2.17299

In [47]:
rimg = app.draw_on(np.asarray(face_image),[face_info])

# draw the image containing the control net key points
face_kps = draw_kps(face_image, face_info['kps'])

f, axarr = plt.subplots(1,2,figsize=(10,10))
axarr[0].imshow(rimg); axarr[0].axis('off')
axarr[1].imshow(face_kps); axarr[1].axis('off');

AttributeError: 'str' object has no attribute 'bbox'